In [1]:
import pandas as pd
from sqlalchemy import create_engine

# 1. Creat the initial DB (clean and load)

## 1.1 Prepare Migration Dataset using pandas

### 1.1.1 Read the data

In [2]:
excel_file = "Resources/Turkey vultures in North and South America - migration.xlsx"
vulture_data_df = pd.read_excel(excel_file, index_col=None)
vulture_data_df.head(2)

,event-id,visible,timestamp,location-long,location-lat,manually-marked-outlier,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,...,animal-life-stage,animal-mass,attachment-type,deployment-comments,deployment-id,duty-cycle,study-site,tag-manufacturer-name,tag-mass,tag-model
0,283203879,True,2003-11-14 16:00:00.000,-75.39717,40.48933,False,gps,Cathartes aura,42500,Butterball,...,adult,2372.0,harness,trapped in Pennsylvania using padded-leg hold ...,42500-Butterball,1 fix per hour,East Coast of North America,Microwave Telemetry,70,PTT100
1,283203880,True,2003-11-14 17:00:00.000,-75.39717,40.48933,False,gps,Cathartes aura,42500,Butterball,...,adult,2372.0,harness,trapped in Pennsylvania using padded-leg hold ...,42500-Butterball,1 fix per hour,East Coast of North America,Microwave Telemetry,70,PTT100


In [3]:
vulture_data_df.keys()
# vulture_data_df.columns

Index(['event-id', 'visible', 'timestamp', 'location-long', 'location-lat',
       'manually-marked-outlier', 'sensor-type',
       'individual-taxon-canonical-name', 'tag-local-identifier',
       'individual-local-identifier', 'study-name', 'utm-easting',
       'utm-northing', 'utm-zone', 'study-timezone', 'study-local-timestamp',
       'tag-id', 'animal-id', 'animal-taxon', 'deploy-on-date',
       'deploy-off-date', 'animal-comments', 'animal-life-stage',
       'animal-mass', 'attachment-type', 'deployment-comments',
       'deployment-id', 'duty-cycle', 'study-site', 'tag-manufacturer-name',
       'tag-mass', 'tag-model'],
      dtype='object')

### 1.1.2 Select columns, and change column names

In [4]:
# Select columns 
new_vulture_data_df = vulture_data_df[['event-id', 'timestamp', 'location-long', 'location-lat','individual-taxon-canonical-name', 'tag-local-identifier',
       'individual-local-identifier']].copy()
new_vulture_data_df.head()

,event-id,timestamp,location-long,location-lat,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier
0,283203879,2003-11-14 16:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
1,283203880,2003-11-14 17:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
2,283203881,2003-11-14 18:00:00.000,-75.33317,40.32467,Cathartes aura,42500,Butterball
3,283203882,2003-11-14 19:00:00.000,-75.35617,40.33983,Cathartes aura,42500,Butterball
4,283203883,2003-11-14 20:00:00.000,-75.42650,40.31550,Cathartes aura,42500,Butterball


In [5]:
#????? change column name ('-' is not compatible in a lot of system)
#also change the column name of the following so consisten with the other info datasets 'individual_taxon_canonical_name', 'tag_local_identifier','individual_local_identifier'to 'animal_taxon','tag_id', 'animal_id'

new_vulture_data_df.columns = ['event_id', 'timestamp', 'location_long', 'location_lat',
                               'animal_taxon', 'tag_id','animal_id']
new_vulture_data_df.head()

,event_id,timestamp,location_long,location_lat,animal_taxon,tag_id,animal_id
0,283203879,2003-11-14 16:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
1,283203880,2003-11-14 17:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
2,283203881,2003-11-14 18:00:00.000,-75.33317,40.32467,Cathartes aura,42500,Butterball
3,283203882,2003-11-14 19:00:00.000,-75.35617,40.33983,Cathartes aura,42500,Butterball
4,283203883,2003-11-14 20:00:00.000,-75.42650,40.31550,Cathartes aura,42500,Butterball


### 1.1.3 Clean DataFrame: drop NA, filter

In [6]:
new_vulture_data_df.count()

event_id         220077
timestamp        220077
location_long    220077
location_lat     220077
animal_taxon     220077
tag_id           220077
animal_id        220077
dtype: int64

In [7]:
# drop rows without long and lat
new_vulture_data_df = new_vulture_data_df.dropna(how="any")
new_vulture_data_df.count()

event_id         220077
timestamp        220077
location_long    220077
location_lat     220077
animal_taxon     220077
tag_id           220077
animal_id        220077
dtype: int64

In [8]:
# filter data to only keep turkey vulture (Cathartes aura) data
new_vulture_data_df = new_vulture_data_df.loc[new_vulture_data_df
                                              ['animal_taxon'] == "Cathartes aura", :]
new_vulture_data_df.count()

event_id         220077
timestamp        220077
location_long    220077
location_lat     220077
animal_taxon     220077
tag_id           220077
animal_id        220077
dtype: int64

In [9]:
# new_vulture_data_df.set_index("event_id")

In [10]:
new_vulture_data_df.dtypes

event_id           int64
timestamp         object
location_long    float64
location_lat     float64
animal_taxon      object
tag_id             int64
animal_id         object
dtype: object

## 1.2 Prepare Vulture Info Dataset using pandas

In [11]:
# Read data
csv_file = "Resources/Turkey vultures in North and South America-reference-data.csv"
vulture_info_df = pd.read_csv(csv_file, low_memory=False)
vulture_info_df.head()

,tag-id,animal-id,animal-taxon,deploy-on-date,deploy-off-date,animal-comments,animal-life-stage,animal-mass,attachment-type,deployment-comments,deployment-id,duty-cycle,study-site,tag-manufacturer-name,tag-mass,tag-model
0,42500,Butterball,Cathartes aura,2003-11-14 16:00:00.000,2004-03-14 20:00:01.000,migratory,adult,2372.0,harness,trapped in Pennsylvania using padded-leg hold ...,42500-Butterball,1 fix per hour,East Coast of North America,Microwave Telemetry,70.0,PTT100
1,52067,Irma,Cathartes aura,2004-09-06 17:00:00.000,2013-03-18 22:00:01.000,non-migratory,adult,2012.0,harness,trapped in Pennsylvania using padded-leg hold ...,52067-Irma,1 fix per hour,East Coast of North America,Microwave Telemetry,70.0,PTT100
2,42500,Schaumboch,Cathartes aura,2004-10-08 15:00:00.000,2006-03-29 17:00:01.000,migratory,adult,1951.0,harness,trapped in Pennsylvania using padded-leg hold ...,42500-Schaumboch,1 fix per hour,East Coast of North America,Microwave Telemetry,70.0,PTT100
3,52069,Disney,Cathartes aura,2004-10-11 14:00:00.000,2011-10-18 23:00:01.000,migratory,adult,2108.0,harness,trapped in Pennsylvania using padded-leg hold ...,52069-Disney,1 fix per hour,East Coast of North America,Microwave Telemetry,70.0,PTT100
4,57954,Prado,Cathartes aura,2005-11-02 15:00:00.000,2009-07-07 00:00:01.000,non-migratory,adult,1710.0,harness,trapped in California using walk-in traps,57954-Prado,1 fix per hour,West Coast of North America,Microwave Telemetry,70.0,PTT100


In [12]:
# Get column names
vulture_info_df.columns

Index(['tag-id', 'animal-id', 'animal-taxon', 'deploy-on-date',
       'deploy-off-date', 'animal-comments', 'animal-life-stage',
       'animal-mass', 'attachment-type', 'deployment-comments',
       'deployment-id', 'duty-cycle', 'study-site', 'tag-manufacturer-name',
       'tag-mass', 'tag-model'],
      dtype='object')

In [13]:
# Select columns 
new_vulture_info_df = vulture_info_df[['tag-id', 'animal-id', 'animal-taxon', 'deploy-on-date',
       'deploy-off-date', 'animal-comments', 'animal-life-stage',
       'animal-mass',  'deployment-comments',
        'study-site']].copy()
new_vulture_info_df.head(1)

,tag-id,animal-id,animal-taxon,deploy-on-date,deploy-off-date,animal-comments,animal-life-stage,animal-mass,deployment-comments,study-site
0,42500,Butterball,Cathartes aura,2003-11-14 16:00:00.000,2004-03-14 20:00:01.000,migratory,adult,2372.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America


In [14]:
# Change column names ('-' to '_')
new_vulture_info_df.columns = ['tag_id', 'animal_id', 'animal_taxon', 'deploy_on_date',
       'deploy_off_date', 'animal_comments', 'animal_life_stage',
       'animal_mass',  'deployment_comments',
        'study_site']
new_vulture_info_df.head()

,tag_id,animal_id,animal_taxon,deploy_on_date,deploy_off_date,animal_comments,animal_life_stage,animal_mass,deployment_comments,study_site
0,42500,Butterball,Cathartes aura,2003-11-14 16:00:00.000,2004-03-14 20:00:01.000,migratory,adult,2372.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America
1,52067,Irma,Cathartes aura,2004-09-06 17:00:00.000,2013-03-18 22:00:01.000,non-migratory,adult,2012.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America
2,42500,Schaumboch,Cathartes aura,2004-10-08 15:00:00.000,2006-03-29 17:00:01.000,migratory,adult,1951.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America
3,52069,Disney,Cathartes aura,2004-10-11 14:00:00.000,2011-10-18 23:00:01.000,migratory,adult,2108.0,trapped in Pennsylvania using padded-leg hold ...,East Coast of North America
4,57954,Prado,Cathartes aura,2005-11-02 15:00:00.000,2009-07-07 00:00:01.000,non-migratory,adult,1710.0,trapped in California using walk-in traps,West Coast of North America


In [15]:
new_vulture_info_df.count()

tag_id                 19
animal_id              19
animal_taxon           19
deploy_on_date         19
deploy_off_date        19
animal_comments        19
animal_life_stage      19
animal_mass            12
deployment_comments    19
study_site             19
dtype: int64

## 1.3 Create and Load to DB

### 1.3.1 Connect to local database

In [16]:
database_path = "vulture_etl"
engine = create_engine(f"sqlite:///{database_path}")

### 1.3.2 Check for tables and creat new tables

In [17]:
engine.table_names()

# ??? SQLite supports a limited subset of ALTER TABLE, add primary keys (https://stackoverflow.com/questions/969187/altering-sqlite-column-type-and-adding-pk-constraint)
# trying to create a table with primary key then load it. But the load part (pd.to_sql) keep giving error

engine.execute('CREATE TABLE IF NOT EXISTS "test" ('
                'event_id INTEGER NOT NULL,'
               'timestamp VARCHAR,'
               'location_long DECIMAL(3,5),'
               'location_lat DECIMAL(3,5),'
                'animal_taxon VARCHAR,'
               'tag_id INTEGER NOT NULL,'
               'animal_id VARCHAR,'
               'UNIQUE (event_id),'
               'PRIMARY KEY (event_id));')

pd.read_sql_query('select * from test', con=engine).head()



# ????if else to prevent build twice
# ??? set foreign key

,event_id,timestamp,location_long,location_lat,animal_taxon,tag_id,animal_id
0,283203879,2003-11-14 16:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
1,283203880,2003-11-14 17:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
2,283203881,2003-11-14 18:00:00.000,-75.33317,40.32467,Cathartes aura,42500,Butterball
3,283203882,2003-11-14 19:00:00.000,-75.35617,40.33983,Cathartes aura,42500,Butterball
4,283203883,2003-11-14 20:00:00.000,-75.42650,40.31550,Cathartes aura,42500,Butterball


### 1.3.3 Use pandas to load csv converted DataFrame into database

In [18]:
new_vulture_data_df.to_sql(name='test', con=engine, if_exists='replace', index=False)

In [19]:
new_vulture_info_df.to_sql(name='vulture_detail', con=engine, if_exists='append', index=False)

In [20]:
# pd.read_sql_query('SELECT sql FROM sqlite_master WHERE name='migration_paths')
                

In [21]:
# engine.execute('alter table migration_paths add primary key(event_id)')

In [22]:
# with engine.connect() as con:
#     con.execute('ALTER TABLE `migration_paths` ADD PRIMARY KEY (`event-id`);')
    
# con = sqlalchemy.create_engine(url, client_encoding='utf8')
# engine.execute('alter table test add primary (event-id);')

In [23]:
pd.read_sql_query('select * from test', con=engine).head()

,event_id,timestamp,location_long,location_lat,animal_taxon,tag_id,animal_id
0,283203879,2003-11-14 16:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
1,283203880,2003-11-14 17:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
2,283203881,2003-11-14 18:00:00.000,-75.33317,40.32467,Cathartes aura,42500,Butterball
3,283203882,2003-11-14 19:00:00.000,-75.35617,40.33983,Cathartes aura,42500,Butterball
4,283203883,2003-11-14 20:00:00.000,-75.42650,40.31550,Cathartes aura,42500,Butterball


In [24]:
pd.read_sql_query('select * from test', con=engine).count()

event_id         220077
timestamp        220077
location_long    220077
location_lat     220077
animal_taxon     220077
tag_id           220077
animal_id        220077
dtype: int64

In [25]:
new_vulture_data_df.to_sql(name='test', con=engine, if_exists='append', index=False)

In [26]:
pd.read_sql_query('select * from test', con=engine).head()

,event_id,timestamp,location_long,location_lat,animal_taxon,tag_id,animal_id
0,283203879,2003-11-14 16:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
1,283203880,2003-11-14 17:00:00.000,-75.39717,40.48933,Cathartes aura,42500,Butterball
2,283203881,2003-11-14 18:00:00.000,-75.33317,40.32467,Cathartes aura,42500,Butterball
3,283203882,2003-11-14 19:00:00.000,-75.35617,40.33983,Cathartes aura,42500,Butterball
4,283203883,2003-11-14 20:00:00.000,-75.42650,40.31550,Cathartes aura,42500,Butterball


# 2. Load New Data from another Source (Acopian Center)

## 2.1 Migration path data

### 2.1.1 Read the data

In [31]:
csv_file = "Resources/Vultures Acopian Center USA 2003-2016-reference-data.csv"
v_data_df = pd.read_csv(csv_file, index_col=None, low_memory=False)
v_data_df.head(2)

,tag-id,animal-id,animal-taxon,deploy-on-date,deploy-off-date,animal-comments,animal-death-comments,animal-life-stage,animal-mass,animal-sex,...,deployment-id,duty-cycle,manipulation-type,study-site,tag-manufacturer-name,tag-mass,tag-model,tag-production-date,tag-readout-method,tag-serial-no
0,42500,Butterball,Cathartes aura,2003-11-05 00:00:00.000,2004-04-19 23:59:00.000,migratory,NaN,adult,2372.0,m,...,42500-Butterball,1 fix per hour,none,Eastern USA,Microwave Telemetry,70.0,PTT100,NaN,satellite,NaN
1,52067,Irma,Cathartes aura,2004-09-06 00:00:00.000,NaN,non-migratory,NaN,adult,2012.0,,...,52067-Irma,1 fix per hour,none,Eastern USA,Microwave Telemetry,70.0,PTT100,NaN,satellite,NaN


In [32]:
v_data_df.keys()
# vulture_data_df.columns

Index(['tag-id', 'animal-id', 'animal-taxon', 'deploy-on-date',
       'deploy-off-date', 'animal-comments', 'animal-death-comments',
       'animal-life-stage', 'animal-mass', 'animal-sex', 'animal-taxon-detail',
       'attachment-type', 'deploy-on-person', 'deployment-comments',
       'deployment-end-comments', 'deployment-end-type', 'deployment-id',
       'duty-cycle', 'manipulation-type', 'study-site',
       'tag-manufacturer-name', 'tag-mass', 'tag-model', 'tag-production-date',
       'tag-readout-method', 'tag-serial-no'],
      dtype='object')

## 2.1.2 Select columns, and change column names

In [30]:
# Select columns 
new_v_data_df = v_data_df[['event-id', 'timestamp', 'location-long', 'location-lat','individual-taxon-canonical-name', 'tag-local-identifier',
       'individual-local-identifier']].copy()
new_v_data_df.head()

KeyError: "['event-id' 'timestamp' 'location-long' 'location-lat'\n 'individual-taxon-canonical-name' 'tag-local-identifier'\n 'individual-local-identifier'] not in index"

In [ ]:
#????? change column name ('-' is not compatible in a lot of system)
#also change the column name of the following so consisten with the other info datasets 'individual_taxon_canonical_name', 'tag_local_identifier','individual_local_identifier'to 'animal_taxon','tag_id', 'animal_id'

new_vulture_data_df.columns = ['event_id', 'timestamp', 'location_long', 'location_lat',
                               'animal_taxon', 'tag_id','animal_id']
new_vulture_data_df.head()

### 2.1.3 Clean DataFrame: drop NA, filter

In [ ]:
new_vulture_data_df.count()

In [ ]:
# drop rows without long and lat
new_vulture_data_df = new_vulture_data_df.dropna(how="any")
new_vulture_data_df.count()

In [ ]:
# filter data to only keep turkey vulture (Cathartes aura) data
new_vulture_data_df = new_vulture_data_df.loc[new_vulture_data_df
                                              ['animal_taxon'] == "Cathartes aura", :]
new_vulture_data_df.count()

In [ ]:
# new_vulture_data_df.set_index("event_id")

In [ ]:
new_vulture_data_df.dtypes

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [ ]:
pd.read_sql_query('select * from migration_paths', con=engine).head()

### Confirm data has been added by querying the customer_location table

In [ ]:
pd.read_sql_query('select * from vulture_detail', con=engine).head()

In [ ]:
engine.table_names()

In [ ]:
conn = engine.connect()

from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

session = Session(bind=engine)
Base.metadata.create_all(engine)

In [ ]:

session.commit()

In [ ]:
names = session.query(migration_paths)
for name in names:
    
    print(migration_paths.animal-id)

## Data from Acopian Center

# list of cities

In [ ]:
from citipy import citipy


In [ ]:
lats = new_vulture_data_df['location-lat'].values.tolist()
lngs = new_vulture_data_df['location-long'].values.tolist()
type(lngs)

In [ ]:
lat_lngs = []
cities = []

In [ ]:
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    cities.append(city)



In [ ]:
len(cities)
cities[-1]

In [ ]:
cities_df = new_vulture_data_df[['event-id', 'location-lat', 'location-long']]
cities_df["cities"] = pd.DataFrame({"cities" : cities})
cities_df[['event-id', 'location-lat', 'location-long', 'cities']]
